In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
import time

In [2]:
scrapping_results={} # create a dictionary to hold all results for mongodb

# NASA Mars News

In [3]:
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)

In [4]:
nasa_url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(nasa_url)
html=browser.html
soup=bs(html,'html.parser')

In [5]:
latest_news_title=soup.find('div',{"class":"content_title"}).a.text.strip()

In [6]:
print(latest_news_title)

NASA Invites Students to Name Mars 2020 Rover


In [7]:
scrapping_results['latest_news_title']=latest_news_title

In [8]:
latest_news_p=soup.find('div',{"class":"rollover_description_inner"}).text # have to choose this tag, <div class="article_teaser_body"> retuns a notype error

In [9]:
print(latest_news_p)

Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


In [10]:
browser.quit()

In [11]:
scrapping_results['latest_news_p']=latest_news_p

# JPL Mars Featured Image

In [12]:
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)

In [13]:
jpl_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
browser.click_link_by_partial_text('FULL IMAGE') # click on FULL IMAGE to go to next page 
time.sleep(5) # wait for 5 seconds until everything is loaded into the 2nd page
browser.click_link_by_partial_text('more info') #click on 'more info' to get the image of original size
html=browser.html
soup=bs(html,'html.parser')

In [14]:
result=soup.find('figure',{'class':'lede'}).a['href']

In [15]:
print(result)

/spaceimages/images/largesize/PIA17896_hires.jpg


In [16]:
jpl_base_url='https://www.jpl.nasa.gov/'
jpl_featured_image_url=jpl_base_url+result
print(jpl_featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA17896_hires.jpg


In [17]:
scrapping_results['jpl_img']=jpl_featured_image_url

In [18]:
browser.quit()

# Mars Weather

In [19]:
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)
twitter_url='https://twitter.com/MarsWxReport'
browser.visit(twitter_url)
html=browser.html
soup=bs(html,'html.parser')

In [20]:
# since the latest tweet is not always about weather, extract all tweets and use list comprehension to store them in a list
results=soup.find_all('p',{'class':'TweetTextSize'})
tweet_list=[tweet.text.strip() for tweet in results] 

In [21]:
# tweets of mars weather always start with 'InSight sol', loop through all tweets, if 'Insight sol' is in a tweet, store it (it's the lastest tweet of mars weather) in a variable and stop the loop immediately
for tweet in tweet_list:
    if ('InSight sol' in tweet):
        mars_weather=tweet.lstrip('InSight ')
        break
print(mars_weather)

sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPapic.twitter.com/9YLawm67zS


In [22]:
scrapping_results['latest_tweet']=mars_weather

In [23]:
browser.quit()

# Mars Facts

In [24]:
mars_facts_url='https://space-facts.com/mars/'

In [25]:
table=pd.read_html(mars_facts_url)

In [26]:
df=table[0]
html_table=df.to_html()
mars_facts_html_table=html_table.replace('\n','')
mars_facts_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars - Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>1</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>2</th>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>3</th>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>4</th>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>5</th>      <td>Temperature:</td>      <td>-153 to 20 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [27]:
scrapping_results['table']=mars_facts_html_table

# Mars Hemispheres

In [28]:
executable_path={'executable_path':'/usr/local/bin/chromedriver'}
browser=Browser('chrome',**executable_path,headless=False)
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [29]:
titles=soup.find_all('h3') #all titles in <h3>tag
title_list=[title.text for title in titles] # extract titles and save to a list
print(title_list)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']


In [30]:
usgs_base_url='https://astrogeology.usgs.gov'
next_page_urls=soup.find_all('div',{'class':'description'}) # extract the links to the 2nd page where original img link can be scrapped
next_page_full_url_list=[usgs_base_url+url.a['href'] for url in next_page_urls] 
print(next_page_full_url_list)

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


In [31]:
original_image_url_list=[]

In [32]:
# use loop to visit the 2nd page of each image
for url in next_page_full_url_list:
    browser.visit(url)
    html=browser.html
    soup=bs(html,'html.parser')
    url=soup.find_all('li') # returns a list
    original_image_url_list.append(url[1].a['href']) # there are 2 two links, sample and orginal. the link to original is the 2nd element in the list
print(original_image_url_list)

['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif']


In [33]:
browser.quit()

In [35]:
hemisphere_image_urls=[dict(title=title_list[i].replace(' Enhanced',''), img_url=original_image_url_list[i]) for i in range(4)]
# dict() is a function to create a dictionary
# can't use for loop and list.append, cuz it results in a list of the same elements


<class 'list'>


In [38]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]


In [36]:
scrapping_results['hemisphere']=hemisphere_image_urls

In [37]:
print(scrapping_results)

{'latest_news_title': 'NASA Invites Students to Name Mars 2020 Rover', 'latest_news_p': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.", 'jpl_img': 'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA17896_hires.jpg', 'latest_tweet': 'sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)\nwinds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)\npressure at 7.50 hPapic.twitter.com/9YLawm67zS', 'table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars - Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>1</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>2</th>      <td>Moons:</td>      <td>2</td>      <td>1